# Measuring memory overhead using getsizeof

In [1]:
from collections import namedtuple, deque, OrderedDict
from sys import getsizeof
import array

## Empty data structures

In [6]:
getsizeof(1)

28

In [7]:
getsizeof(())

56

In [8]:
Point = namedtuple('Point', 'x y')

In [9]:
p = Point(1, 2)
p

Point(x=1, y=2)

In [10]:
p.x

1

In [11]:
p.y

2

In [12]:
p[0]

1

In [13]:
Foo = namedtuple('Foo', '')
getsizeof(Foo())

56

In [14]:
getsizeof(p)

72

In [17]:
getsizeof([])

72

In [19]:
getsizeof(deque())

640

In [20]:
getsizeof({})

248

In [21]:
getsizeof(set())

232

In [22]:
getsizeof(OrderedDict())

312

In [23]:
getsizeof(b'')

33

In [24]:
getsizeof(u'')

49

In [25]:
getsizeof(object())

16

In [26]:
getsizeof([1])

80

In [27]:
getsizeof(1)

28

In [28]:
getsizeof(1.1)

24

In [29]:
getsizeof('')

49

In [30]:
getsizeof(array.array('i', []))

64

In [32]:
getsizeof(array.array('i', [1]*100))

464

In [44]:
getsizeof([1] * 100)

872

In [45]:
getsizeof([
    {i:i, 'foo': 'bar'} 
    for i in range(100)
])

920

In [46]:
getsizeof([[] for i in range(100)])

920

In [47]:
getsizeof(array.array('l', [1] * 100))

864

In [48]:
getsizeof(array.array('I', [1] * 100))

464

In [49]:
getsizeof(array.array('b', [1] * 100))

164

In [50]:
import numpy

In [51]:
getsizeof(numpy.array([1] * 100, numpy.int8))

196

In [52]:
getsizeof(numpy.array([1.1] * 100))

896

In [53]:
import random
lst = [random.random() for i in range(1024)]

In [54]:
%timeit lst1 = [ 2.0 * x for x in lst ]

54.9 µs ± 4.42 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [56]:
arr = array.array('d', lst)

In [57]:
%timeit arr1 = [ 2.0 * x for x in arr ]

55.5 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [58]:
arr2 = array.array('d', [0] * len(lst))

In [60]:
%%timeit
for i in range(len(lst)):
    arr2[i] = arr[i] * 2

161 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [61]:
narr = numpy.array(lst)

In [62]:
%timeit narr1 = 2.0 * narr

1.28 µs ± 20.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [68]:
class Point:
    
    def __init__(self, x, y):
        self.x, self.y = x, y
        
Point2 = namedtuple('Point2', 'x y')

In [69]:
p = Point(1, 2)
getsizeof(p)

64

In [70]:
getsizeof(Point2(1, 2))

72

In [73]:
getsizeof(p.__dict__)

120

# Optimizing class size with `__slots__`

In [89]:
class Foo(object):
    pass

class FooSlots(object):
    __slots__ = ()

In [90]:
getsizeof(Foo())

64

In [91]:
getsizeof(FooSlots())

16

In [92]:
FooNT = namedtuple('Foo', 'a b')

class FooSlots(object):
    __slots__ = ('a', 'b')
    def __init__(self, a, b):
        self.a, self.b = a, b

In [95]:
getsizeof(FooNT(1, 1))

72

In [96]:
getsizeof(FooSlots(1, 2))

64

In [97]:
foo = FooSlots(1, 2)
foo.z = 5

AttributeError: 'FooSlots' object has no attribute 'z'

If you *don't* define `__slots__`, then an instance `__dict__` will be created, even if you inherit from a class that defines `__slots__`

In [101]:
class ASlots(object):
    __slots__ = ('a',)
    
class BSlots(ASlots):
    __slots__ = ('b',)
    
class CSlots(BSlots):
    pass


In [102]:
cs = CSlots()
cs.__dict__

{}

In [103]:
cs.a = 5

In [104]:
cs.__dict__

{}

In [106]:
cs.c = 5
cs.__dict__

{'c': 5}

In [107]:
bs = BSlots()
bs.__dict__

AttributeError: 'BSlots' object has no attribute '__dict__'

In [108]:
bs.a = 5
bs.b = 10

In [109]:
bs.c = 5

AttributeError: 'BSlots' object has no attribute 'c'

# Time/space

## Vectors

Implementation: packed array of values [with some metadata]

Examples:

- list
- tuple
- deque
- class/object attributes (when using `__slots__`)

Performance:

- lookup by index is O(1)
- test for membership is O(n) (`in`, `.find()`)
- "end" modifications are O(1) (_amortized_)
- "insert" modifications are O(n)

## Heaps

Implementation: half-sorted balanced binary tree

Example: `heapq` standard library module

Performance:

- test for membership is O(n)
- modifications are O(log n)

## Hashtables

Examples:

- set
- dict
- class/object attributes (unless using `__slots__`)

Performance:

- testing, lookup, and modification are all O(1) speed (modification is _amortized_ O(1))
- uses more space than a comparable vector

In [110]:
class KeyObj(object):
    """This is a hashable object whose hash value can change at runtime.
    
    That's a no-no.
    """
    def __init__(self, foo):
        self.foo = foo
        
    def __hash__(self):
        return hash(self.foo)

a = KeyObj('foo')
hash(a)

-2214063170088483681

In [111]:
a.foo = 'bar'
hash(a)

4121284657815977373

In [112]:
d = {a: 1}

In [114]:
a in d

True

In [115]:
d[a]

1

In [113]:
a.foo

'bar'

In [116]:
a.foo = 'baz'

In [117]:
d

{<__main__.KeyObj at 0x114d79190>: 1}

(grow the dictionary to force a re-hashing of values)

In [121]:
d.update((i, i) for i in range(1000))

In [122]:
a in d

False

In [123]:
d[a]

KeyError: <__main__.KeyObj object at 0x114d79190>

In [124]:
a.foo = 'bar'

In [125]:
a in d

True

In [126]:
d[a]

1

# Exercise: Time complexity

Use the `random` module and the `%timeit` magic function to compare testing for membership in a set, list, and dict, for different sized data:

In [127]:
import random

def test_data(N):
    lst = list(range(N))
    set_ = set(lst)
    dct = dict(zip(lst, lst))
    return lst, set_, dct

lst100, set100, dict100 = test_data(100)
lst200, set200, dict200 = test_data(200)
lst400, set400, dict400 = test_data(400)
lst1000, set1000, dict1000 = test_data(1000)



In [128]:
import random

def finder(ds):
    return random.randint(0, len(ds)-1) in ds

In [133]:
%timeit finder(lst100)

2.34 µs ± 100 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [132]:
%timeit finder(dict100)

1.52 µs ± 15.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [ ]:
import random

def finder(ds):
    return random.randint(0, len(ds)-1) in ds

In [ ]:
%timeit finder(lst100)

In [ ]:
%timeit finder(lst200)

In [ ]:
%timeit finder(lst400)

In [ ]:
%timeit finder(lst1000)

In [ ]:
%timeit finder(set100)

In [ ]:
%timeit finder(set200)

In [ ]:
%timeit finder(set400)

In [ ]:
%timeit finder(set1000)

In [ ]:
%timeit finder(dict1000)